## Parallelization

Most algorithms in our framework are population based. Therefore, the evaluation function receives not only one but multiple solutions at a time. The amount of individuals depends on the algorithm logic and population size.
The function targets to set objective and constraint values into a matrix and the evaluation can be done independent of *pymoo*. In the following different implementations of the evaluation function are discussed.

### Vectorized

By default matrix and vector based operations are used to speed up calculations. An example of a vectorized evaluation is already provided in our [Getting Started Guide](../getting_started.ipynb).

In [3]:
import autograd.numpy as anp

from pymoo.model.problem import Problem

class MyProblem(Problem):

    def __init__(self, **kwargs):
        super().__init__(n_var=10, n_obj=1, n_constr=0, xl=-5, xu=5, **kwargs)

    def _evaluate(self, x, out, *args, **kwargs):
         out["F"] = anp.sum(x ** 2, axis=1) 

By default no parallelization is applied and only the code is executed.

In [4]:
problem = MyProblem(parallelization = None)

Create some sample input:

In [5]:
import numpy as np

X = np.random.random((10000, problem.n_var))

In [6]:
%timeit F = problem.evaluate(X)

223 µs ± 13 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Multiple Threads 

Another way of defining a problem is by using the `elementwise_evaluation` and then execute each evaluation is a different thread. This can be automatically achieved by the following code:

In [7]:
class MyProblem(Problem):

    def __init__(self, **kwargs):
        super().__init__(n_var=10, n_obj=1, n_constr=0, xl=-5, xu=5, 
                         elementwise_evaluation=True, **kwargs)

    def _evaluate(self, x, out, *args, **kwargs):
         out["F"] = (x ** 2).sum()

In [8]:
problem = MyProblem(parallelization = ("threads", 4))

In [9]:
%timeit F = problem.evaluate(X)

147 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Please note, that for small examples as shown here vectorization will always be faster. However, problem evaluation functions which are based on simulations can often not be easily vectorized because of their independent behavior.

**NOTE:** If your are a Windows user your function call of the minimize method needs to happen in a main method. The reason is explained [here](https://stackoverflow.com/questions/24374288/where-to-put-freeze-support-in-a-python-script) and an example can be found [here](https://github.com/msu-coinlab/pymoo/issues/23). Otherwise, you will get an error message saying to call freeze_support() somewhere in your code.

### Generic Starmap Interface

Another way of parallelized evaluation is to provide an external function that provides the "starmap" interface,
as in the Python standard library `multiprocessing.Pool.starmap` [function](https://docs.python.org/3/library/multiprocessing.html?highlight=multiprocessing#multiprocessing.pool.Pool.starmap).

**NOTE:** For this example the problem is supposed to be of type `elementwise_evaluation=True`.

In [10]:
import multiprocessing

pool = multiprocessing.Pool()
problem = MyProblem(parallelization = ('starmap', pool.starmap))

In [11]:
%timeit problem.evaluate(X)

94.3 ms ± 15.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
pool.close()

### Dask

More advanced is to distribute the evaluation function to a couple of workers. There exists a couple of framework that support the distribution of code. For our framework, we recommend using [Dask](https://dask.org).

A documentation to setup the cluster is available [here](https://docs.dask.org/en/latest/setup/cli.html). Basically, you first start a scheduler somewhere and then connect workers to it. Then, a client object connects to the scheduler and distributes the jobs automatically for you.

In [22]:
from dask.distributed import Client
client = Client()

In [23]:
import numpy as np
from pymoo.model.problem import Problem

# define the evaluation function that directly returns the objective and/or
# constraint values. Please note it must be elementwise_evaluation.
def fun(x):
    return {
        "F": np.sum(x ** 2)
    }

# define a problem without any evaluation function - everything is done by parallelization
class MyProblem(Problem):

    def __init__(self, *args, **kwargs):
        super().__init__(n_var=10, n_obj=1, n_constr=0, xl=-5, xu=5, 
                         elementwise_evaluation=True, *args, **kwargs)

# create the problem and set the parallelization to dask
problem = MyProblem(parallelization=("dask", client, fun))

X = np.random.random((10, problem.n_var))

In [24]:
%timeit problem.evaluate(X)

37.4 ms ± 7.88 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Manually use Dask**

In order to have full control over the distribution you can use Dask manually in the _evaluate function. By using the *parallelization* this is done internally, however, it might be useful to have full control over it.

In [25]:
class MyProblem(Problem):

    def __init__(self, *args, **kwargs):
        super().__init__(n_var=10, n_obj=1, n_constr=0, xl=-5, xu=5, 
                         elementwise_evaluation=True, *args, **kwargs)

    def _evaluate(self, X, out, *args, **kwargs):
        
        def fun(x):
            return np.sum(x ** 2) 
        
        jobs = [client.submit(fun, x) for x in X]
        out["F"] = np.row_stack([job.result() for job in jobs])
        

problem = MyProblem()

In [26]:
%timeit problem.evaluate(X)

485 ms ± 45.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Do not forget to close the client if it is running locally!

In [27]:
client.close()